## Implement the Apriori algorithm and use it to mine category sets that are frequent in the input data

In [339]:
with open('categories.txt', 'r') as f:
    data = f.read().split('\n')
data = [l.split(';') for l in data]
data[:5]

[['Breakfast & Brunch', 'American (Traditional)', 'Restaurants'],
 ['Sandwiches', 'Restaurants'],
 ['Local Services', 'IT Services & Computer Repair'],
 ['Restaurants', 'Italian'],
 ['Food', 'Coffee & Tea']]

### Step 1

Please output all the length-1 frequent categories with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category and should be in the following format:

`support:category`

For example, suppose a category (Fast Food) has an absolute support 3000, then the line corresponding to this frequent category set in "patterns.txt" should be:

`3000:Fast Food`

In [344]:
from collections import defaultdict

def get_C1(D, min_sup:int=None):
    C1 = defaultdict(int)
    for transaction in D:
        for item in transaction:
            C1[frozenset([item])] += 1
    return C1

def get_above_min_sup(C:dict, min_sup:int):
    L_sup = C.copy()
    for key in C:
        if C[key] < min_sup: L_sup.pop(key)
    return L_sup

min_sup = 771
C1 = get_C1(data, min_sup=min_sup)
L1 = get_above_min_sup(C1, min_sup=min_sup)

with open('1/patterns.txt', 'w') as f:
    f.write(
        '\n'.join([f'{v}:{list(k)[0]}' for (k, v) in
                   sorted(L1.items(), key=lambda item: item[1], reverse=True)])
    )

### Step 2

Please write all the frequent category sets along with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category set and should be in the following format:

`support:category_1;category_2;category_3;...`

For example, suppose a category set (Fast Food; Restaurants) has an absolute support 2851, then the line corresponding to this frequent category set in "patterns.txt" should be:

`2851:Fast Food;Restaurants`

**Important Tips**

Make sure that you format each line correctly in the output file. For instance, use a semicolon instead of another character to separate the categories for each frequent category set. 

In the result pattern file, the order of the categories does not matter. For example, the following two cases will be considered equivalent by the grader:

Case 1:

`2851:Fast Food;Restaurants`

Case 2:

`2851:Restaurants;Fast Food `

In [345]:
from itertools import combinations
from datetime import datetime

def get_union(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def pruning(Ck, Lkprev, length):
    temp = Ck.copy()
    for item in Ck:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in Lkprev):
                temp.remove(item)
                break
    return temp
  
def apriori_gen(Lk, k):
    """Generates candidates and eliminates itemsets that are not frequent
    
    Input
    -----
    Lk : dict
        {<itemset>:<support count>}
    k : int
        itemset 'order'
    
    Output
    ------
    Ck : dict
        candidate itemset
    """
    
    Ck = get_union(Lk, k)
    Ck = pruning(Ck, Lk, k-1)
    
    return Ck

D = list(map(frozenset, data))
L = [None]
L.append(L1)
L_final = L1.copy()
k = 2
while (len(L[k-1]) > 0):
    print(f'{datetime.now()}: k={k}')
    Ck = apriori_gen(list(L[k-1].keys()), k)
    Ct = defaultdict(int)
    for transaction in D:
        for can in Ck:
            if can.issubset(transaction):
                Ct[can] += 1
    print(f'Got len(Ck):{len(Ck)}, len(Ct):{len(Ct)}')
    Lk = get_above_min_sup(Ct, min_sup=min_sup)
    L.append(Lk)
    L_final.update(Lk)
    k += 1

with open('2/patterns.txt', 'w') as f:
    f.write(
        '\n'.join([f'{v}:{";".join(list(k))}' for (k, v) in
                   sorted(L_final.items(), key=lambda item: item[1], reverse=True)])
    )

2021-03-22 09:31:50.126965: k=2
Got len(Ck):1225, len(Ct):598
2021-03-22 09:32:05.164178: k=3
Got len(Ck):8, len(Ct):8
2021-03-22 09:32:05.268834: k=4
Got len(Ck):0, len(Ct):0
